In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
path = kagglehub.dataset_download('iqbalmaulana/indonesian-news-dataset')

print('Data source import complete.')


100%|██████████| 272M/272M [00:06<00:00, 44.6MB/s]

Extracting files...


Data source import complete.


In [2]:
!pip install llama-index-core
!pip install llama-index
!pip install llama-index-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.3 MB/s eta 0:00:00


In [3]:
from getpass import getpass
import pandas as pd
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI
import os

In [4]:
df = pd.read_csv(path+ "/data.csv")

In [5]:
df.head()

,id,source,title,image,url,content,date,embedding,created_at,updated_at,summary
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698528/depo-pl...,"TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00,"[-0.01590039,-0.034130897,0.005732614,-0.01853...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Anggota Komisi VII DPR RI Rofik Hananto menyay...
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698522/jokowi-...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00,"[-0.017608976,-0.021786924,0.01547983,-0.00932...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Presiden Joko Widodo telah memerintahkan Wakil...
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698527/hnw-men...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00,"[0.00841488,-0.023665192,0.006762431,-0.013723...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698540/tim-dok...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00,"[-0.012671886,-0.0039057182,0.019575326,-0.016...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...
4,87,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698536/bamsoet...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00,"[-0.015486176,-0.0125719,-0.0122843925,-0.0343...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Ketua MPR RI Bambang Soesatyo telah diangkat s...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32735 entries, 0 to 32734
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          32735 non-null  int64 
 1   source      32735 non-null  object
 2   title       32735 non-null  object
 3   image       32575 non-null  object
 4   url         32735 non-null  object
 5   content     32294 non-null  object
 6   date        32735 non-null  object
 7   embedding   32735 non-null  object
 8   created_at  32735 non-null  object
 9   updated_at  32735 non-null  object
 10  summary     32725 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.7+ MB


In [7]:
df.duplicated().sum()

0

In [8]:
df.describe(include="all")

,id,source,title,image,url,content,date,embedding,created_at,updated_at,summary
count,32735.000000,32735,32735,32575,32735,32294,32735,32735,32735,32735,32725
unique,NaN,7,32651,26449,32735,32086,31769,32193,3452,3452,32725
top,NaN,kumparan,Pertamina Geothermal Energy Bukukan Pendapatan...,https://statik.tempo.co/data/2023/03/10/id_118...,https://nasional.tempo.co/read/1698528/depo-pl...,Terjun politik bisa jadi menjadi salah satu im...,2023-03-28 00:00:00+00,"[0.0015690419,-0.01674868,-0.00096191745,0.002...",2023-04-06 00:15:11.011187,2023-04-06 00:15:11.011187,Anggota Komisi VII DPR RI Rofik Hananto menyay...
freq,NaN,8579,3,22,1,3,5,264,289,289,1
mean,34081.748770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,18044.068638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,33.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,17723.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,35074.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,49896.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.isnull().sum()

,0
id,0
source,0
title,0
image,160
url,0
content,441
date,0
embedding,0
created_at,0
updated_at,0


In [10]:
df = df.dropna(subset=['content'])
df.isnull().sum()

,0
id,0
source,0
title,0
image,157
url,0
content,0
date,0
embedding,0
created_at,0
updated_at,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32294 entries, 0 to 32734
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          32294 non-null  int64 
 1   source      32294 non-null  object
 2   title       32294 non-null  object
 3   image       32137 non-null  object
 4   url         32294 non-null  object
 5   content     32294 non-null  object
 6   date        32294 non-null  object
 7   embedding   32294 non-null  object
 8   created_at  32294 non-null  object
 9   updated_at  32294 non-null  object
 10  summary     32284 non-null  object
dtypes: int64(1), object(10)
memory usage: 3.0+ MB


In [12]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('token')

In [13]:
llm = OpenAI(model="chatgpt-4o-latest")

In [14]:
messages = [
    ChatMessage(
        role="system", content="Explain what is your model name"
    ),
    ChatMessage(role="user", content="Whats your model name"),
]
resp = llm.chat(messages)
print(resp)

assistant: I am based on OpenAI's GPT-4 model.


In [28]:
systemPrompt = """
You are a news article analyzer capable of determining political bias by analyzing what is being highlighted. Please provide an explanation of why you chose a particular side, citing specific examples from the text that demonstrate its alignment. Do not speak in first person.

Assess the article using the following bias ratings:
Far Left: Strongly biased toward liberal causes. Uses extreme, emotional language, misleading reports, or omits unfavorable information.
Left: Moderately biased toward liberal causes. May use loaded words or omit certain details.
Lean Left: Slight to moderate liberal bias. Generally factual but may favor liberal causes.
Center: Neutral. Few loaded words, well-sourced reporting, and balanced presentation of views.
Lean Right: Slight to moderate conservative bias. Generally factual but may favor conservative causes.
Right: Moderately biased toward conservative causes. May use loaded words, omit unfavorable information, or mislead.
Far Right: Strongly biased toward conservative causes. Uses extreme, emotional language, misleading reports, or omits unfavorable information.

Expected Answer:
Bias Rating: Rate the article and explain your choice with brief examples from the text.
Key Indicators of Bias: Opinionated content, selective facts, extreme language, persuasive tactics, unsupported claims, anonymity, entertainment-based content, or disguised sales pitches.
Conclusion: Summarize findings, mentioning if there were any nuances or mixed signals.
"""

In [29]:
text = "Here is the news: " + df['content'].iloc[4]


In [30]:
print(text)

Here is the news: INFO NASIONAL - Ketua MPR RI sekaligus Ketua Umum Ikatan Motor Indonesia (IMI) dan Wakil Ketua Umum Partai Golkar Bambang Soesatyo diangkat sebagai Ketua Dewan Pembina Harley Davidson Club Indonesia (HDCI) periode 2023-2028.Bamsoet berharap HDCI akan tetap konsisten mempertahankan citra positif yang telah dibangun. Sehingga, dapat menginspirasi klub-klub otomotif lainnya untuk memberikan kontribusi terbaik bagi kehidupan sosial kemasyarakatan."Selamat kepada sahabat saya Ahmad Sahroni yang telah terpilih sebagai Ketua Umum HDCI periode 2023-2028 pada Musyawarah Nasional Luar Biasa tanggal 27-28 Januari 2023 di Bali," kata Bamsoet saat membuka Rapat Kerja Nasional (Rakernas) HDCI dan pelantikan pengurus pusat HDCI periode 2023-2028 di Semarang, Jumat malam, 3 Maret 2023.Bamsoet yakin dan percaya dengan pengalaman dan wawasan organisasi yang luas, kemampuan membangun networking yang handal, serta kecintaan yang tulus pada dunia otomotif, Ahmad Sahroni memiliki segala ku

In [31]:
messages = [
    ChatMessage(
        role="system", content=systemPrompt
    ),
    ChatMessage(role="user", content=text),
]
resp = llm.chat(messages)

In [32]:
print(resp)

assistant: **Bias Rating**: Center

**Key Indicators of Bias**:
- The article primarily reports on the appointment of Bambang Soesatyo as the Chairman of the Advisory Board of Harley Davidson Club Indonesia (HDCI) and the election of Ahmad Sahroni as the Chairman of HDCI. It focuses on the organizational aspects of HDCI and its contributions to society, particularly through social and economic initiatives.
- The language used is neutral and factual, with no loaded or emotional terms. The article highlights the positive contributions of HDCI, such as social humanitarian actions and support for economic development through automotive tourism, but does not delve into any controversial or politically charged topics.
- The article does not show any clear favoritism toward a political ideology or party, despite mentioning that Bambang Soesatyo is a member of the Golkar Party. The focus remains on his role in HDCI and the club's activities rather than his political affiliations or policies.
-